In [104]:
import pandas as pd
import src.scripts as src
import matplotlib.pyplot as plt
%matplotlib inline

In [86]:
df = src.read_from_efs('X_reduced.csv')
df.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,nppes_provider_state,npi,generic_name,bene_count,total_claim_count,total_30_day_fill_count,total_day_supply,total_drug_cost,cost_per_bene,cost_per_claim,exclusion
0,AE,1083814099,LEVOTHYROXINE SODIUM,30.0,40,94.0,2818,607.93,20.264333,15.198250,False
1,AE,1083924617,LEVOTHYROXINE SODIUM,95.0,774,925.8,27229,15603.77,164.250211,20.159910,False
2,AE,1093867137,LEVOTHYROXINE SODIUM,48.0,189,495.0,14850,6604.84,137.600833,34.946243,False
3,AE,1184932204,LEVOTHYROXINE SODIUM,20.0,37,82.0,2440,905.01,45.250500,24.459730,False
4,AE,1255694022,LEVOTHYROXINE SODIUM,10.0,36,38.0,1128,571.02,57.102000,15.861667,False


In [87]:
df_if = df.pop('npi')
state = df.pop('nppes_provider_state')
y = df.pop('exclusion').values

In [88]:
X = pd.get_dummies(df, columns=['generic_name'])
X.head()

,bene_count,total_claim_count,total_30_day_fill_count,total_day_supply,total_drug_cost,cost_per_bene,cost_per_claim,generic_name_ALBUTEROL SULFATE,generic_name_ALENDRONATE SODIUM,generic_name_ALLOPURINOL,...,generic_name_TERAZOSIN HCL,generic_name_TIOTROPIUM BROMIDE,generic_name_TRAMADOL HCL,generic_name_TRAZODONE HCL,generic_name_TRIAMTERENE/HYDROCHLOROTHIAZID,generic_name_VALSARTAN,generic_name_VALSARTAN/HYDROCHLOROTHIAZIDE,generic_name_VENLAFAXINE HCL,generic_name_WARFARIN SODIUM,generic_name_ZOLPIDEM TARTRATE
0,30.0,40,94.0,2818,607.93,20.264333,15.198250,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,95.0,774,925.8,27229,15603.77,164.250211,20.159910,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,48.0,189,495.0,14850,6604.84,137.600833,34.946243,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,20.0,37,82.0,2440,905.01,45.250500,24.459730,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10.0,36,38.0,1128,571.02,57.102000,15.861667,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [ ]:
clf = RandomForestClassifier(n_estimators = 100, n_jobs=-1, class_weight='balanced_subsample')

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

fpr, tpr, threshold = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

# method I: plt
plt.title('Random Forest Classification')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()